In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
import statistics as stat
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample

from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, wtf
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.UPMhelper import UPMalerts

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [20]:
q = 0.9999
qv = 0.999999

#### 1) Collecting Alerts

In [3]:
alerts0 = fixNum(unSMART(pd.read_excel('~/Documents/vmfldr/smarts1_2019JanMay.xlsx')))

-> type fixed, kept 1284 rows, 13 columns


In [4]:
alerts1001 = alerts0.loc[alerts0['AlertCode'] == 1001,['AlertCode','LongText','InstrumentName']]
alerts1001['LongText2'] = alerts1001['LongText'].str.extract('\D+(Price Change.+)Bid.+',expand=True)

alerts1001['min05'] = alerts1001['LongText2'].str.findall('Price\D+(\d+\D+)\sis').apply(lambda x: '5 minutes' in x) * 1
alerts1001['min10'] = alerts1001['LongText2'].str.findall('Price\D+(\d+\D+)\sis').apply(lambda x: '10 minutes' in x) * 1
alerts1001['min60'] = alerts1001['LongText2'].str.findall('Price\D+(\d+\D+)\sis').apply(lambda x: '1 hour' in x) * 1

In [5]:
# alerts0.groupby(['AlertCode',alerts0['Datetime'].dt.month]).count()

In [6]:
regextext2 = 'Price Change\D+(\d+\s[a-z]+)[^\(]+\(([\d.]+)%\)[^\(]+\(([\d.]+)%\)\s*'

alerts1001_rematch = alerts1001['LongText2'].str.extractall(regextext2)
alerts1001_rematch_5m = alerts1001_rematch.loc[alerts1001_rematch[0]=='5 minutes',[1,2]].astype(float).reset_index().set_index('level_0')[[1,2]].rename(columns={1:'val_5m',2:'bm_5m'})
alerts1001_rematch_10m = alerts1001_rematch.loc[alerts1001_rematch[0]=='10 minutes',[1,2]].astype(float).reset_index().set_index('level_0')[[1,2]].rename(columns={1:'val_10m',2:'bm_10m'})
alerts1001_rematch_60m = alerts1001_rematch.loc[alerts1001_rematch[0]=='1 hour',[1,2]].astype(float).reset_index().set_index('level_0')[[1,2]].rename(columns={1:'val_60m',2:'bm_60m'})

alerts1001_parsed = pd.concat([alerts1001['InstrumentName'],alerts1001_rematch_5m, alerts1001_rematch_10m, alerts1001_rematch_60m], axis=1)
alerts1001_mean = alerts1001_parsed.groupby('InstrumentName').mean()/100
alerts1001_count = alerts1001_parsed.groupby('InstrumentName').count()

#### 2) Collecting Data

In [7]:
query = """
        SELECT created, price, quantity, trading_pair
        from order_fill_event
        where created::date between '2019-01-01' and '2019-05-31'
        """

data0 = execute_query_data_frame(query,'engine')

INFO:root:Constructing Singleton
INFO:root:Using global configuration /Users/sqian/msv.yaml.
INFO:root:Completed loading configuration.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:01.081041


INFO:root:Loaded dataframe with 3271817 rows in 0:01:12.157880.


#### 3) UPM Quantiles

In [59]:
data1 = data0.sort_values('created')
data1['time05'] = data1['created'] + datetime.timedelta(minutes=5)
data1['time10'] = data1['created'] + datetime.timedelta(minutes=10)
data1['time60'] = data1['created'] + datetime.timedelta(minutes=60)
table1 = data1[['created','price','trading_pair']]
# table1

In [19]:
table2 = data1.drop(columns=['created','quantity']).rename(columns={'time05':'created','price':'price2'})
df_merge_asof = pd.merge_asof(table1, table2, on='created', by='trading_pair')
df_merge_asof['change'] = abs(df_merge_asof['price2']/df_merge_asof['price']-1)
temp = df_merge_asof.dropna(how='any').groupby('trading_pair').apply(lambda x: x.quantile(q)).sort_values('change')
temp2 = pd.concat([temp,alerts1001_mean['val_5m']],axis=1,sort=False)
temp2['room'] = temp2['change'] - temp2['val_5m']

syntax = (round(temp2['change']*100,2)).clip(0,200).astype(str)+'%'
syntax2 = syntax.loc[syntax != '200.0%'].reset_index();syntax2
syntax2['syntax'] = '{' + syntax2['index'] + ',(' + syntax2['change'] + ',)}'
syntax_05m = '(200%,);'+';'.join(syntax2['syntax'])
syntax_05m

'(200%,);{BTCUSD,(7.62%,)};{ZECUSD,(8.3%,)};{ETHUSD,(8.89%,)};{BCHUSD,(11.56%,)};{ETHBTC,(13.08%,)};{LTCUSD,(15.16%,)};{ZECETH,(21.92%,)};{ZECBTC,(33.42%,)};{LTCBTC,(67.7%,)};{LTCETH,(105.9%,)}'

In [10]:
table2 = data1.drop(columns=['created','quantity']).rename(columns={'time10':'created','price':'price2'})
df_merge_asof = pd.merge_asof(table1, table2, on='created', by='trading_pair')
df_merge_asof['change'] = abs(df_merge_asof['price2']/df_merge_asof['price']-1)
temp = df_merge_asof.dropna(how='any').groupby('trading_pair').apply(lambda x: x.quantile(q)).sort_values('change')
temp2 = pd.concat([temp,alerts1001_mean[['val_10m']]],axis=1,sort=False)
temp2['room'] = temp2['change'] - temp2['val_10m']

syntax = (round(temp2['change']*100,2)).clip(0,200).astype(str)+'%'
syntax2 = syntax.loc[syntax != '200.0%'].reset_index();syntax2
syntax2['syntax'] = '{' + syntax2['index'] + ',(' + syntax2['change'] + ',)}'
syntax_10m = '(200%,);'+';'.join(syntax2['syntax'])
syntax_10m

'(200%,);{ZECUSD,(8.33%,)};{ETHUSD,(9.97%,)};{BTCUSD,(11.67%,)};{BCHUSD,(12.87%,)};{ETHBTC,(13.11%,)};{LTCUSD,(19.06%,)};{ZECETH,(21.92%,)};{ZECBTC,(33.42%,)};{LTCBTC,(67.7%,)};{LTCETH,(105.9%,)}'

In [11]:
table2 = data1.drop(columns=['created','quantity']).rename(columns={'time60':'created','price':'price2'})
df_merge_asof = pd.merge_asof(table1, table2, on='created', by='trading_pair')
df_merge_asof['change'] = abs(df_merge_asof['price2']/df_merge_asof['price']-1)
temp = df_merge_asof.dropna(how='any').groupby('trading_pair').apply(lambda x: x.quantile(q)).sort_values('change')
temp2 = pd.concat([temp,alerts1001_mean[['val_60m']]],axis=1,sort=False)
temp2['room'] = temp2['change'] - temp2['val_60m']

syntax = (round(temp2['change']*100,2)).clip(0,200).astype(str)+'%'
syntax2 = syntax.loc[syntax != '200.0%'].reset_index();syntax2
syntax2['syntax'] = '{' + syntax2['index'] + ',(' + syntax2['change'] + ',)}'
syntax_60m = '(200%,);'+';'.join(syntax2['syntax'])
syntax_60m

'(200%,);{ETHBTC,(11.14%,)};{ZECUSD,(13.08%,)};{ZECBTC,(14.97%,)};{ETHUSD,(17.57%,)};{BTCUSD,(17.98%,)};{BCHUSD,(20.11%,)};{ZECETH,(21.95%,)};{LTCUSD,(22.16%,)};{LTCBTC,(67.75%,)};{LTCETH,(105.9%,)};{BCHBTC,(153.65%,)};{ZECBCH,(165.11%,)}'

#### 4) UVM Quantiles

In [22]:
temp = pd.pivot_table(data0,index='created',columns='trading_pair',values='quantity')

In [37]:
uvm = pd.DataFrame()

for i in temp.columns.tolist():
    temp2 = pd.DataFrame(temp[i].dropna())
    temp2['rolling05'] = temp2[i].rolling('5min').sum()
    temp2['rolling10'] = temp2[i].rolling('10min').sum()
    temp2['rolling60'] = temp2[i].rolling('60min').sum()
    temp3 = pd.DataFrame(temp2.quantile(qv)).transpose()[['rolling05','rolling10','rolling60']]
    temp3.index = [i]
    uvm = uvm.append(temp3.reset_index(),sort=False)

In [3]:
uvm2 = uvm.set_index('index').astype(int).sort_values('rolling05',ascending=False)

NameError: name 'uvm' is not defined

In [25]:
syntax = uvm2['rolling05'].reset_index()
syntax['syntax'] = '{' + syntax['index'] + ',(x' + syntax['rolling05'].astype(str) + ',)}'
syntaxprint = '(x1,);'+';'.join(syntax['syntax'])
syntaxprint

'(x1,);{ETHUSD,(x12756,)};{LTCUSD,(x11712,)};{LTCBTC,(x6962,)};{ZECUSD,(x4108,)};{ETHBTC,(x3601,)};{ZECBTC,(x3366,)};{BCHUSD,(x1434,)};{LTCETH,(x1123,)};{BTCUSD,(x734,)};{BCHBTC,(x700,)};{ZECETH,(x684,)};{ZECBCH,(x424,)};{LTCBCH,(x318,)};{BCHETH,(x224,)};{ZECLTC,(x56,)}'

In [26]:
syntax = uvm2['rolling10'].reset_index()
syntax['syntax'] = '{' + syntax['index'] + ',(x' + syntax['rolling10'].astype(str) + ',)}'
syntaxprint = '(x1,);'+';'.join(syntax['syntax'])
syntaxprint

'(x1,);{ETHUSD,(x19819,)};{LTCUSD,(x15371,)};{LTCBTC,(x9034,)};{ZECUSD,(x5940,)};{ETHBTC,(x4132,)};{ZECBTC,(x3425,)};{BCHUSD,(x1970,)};{LTCETH,(x1310,)};{BTCUSD,(x1201,)};{BCHBTC,(x1267,)};{ZECETH,(x684,)};{ZECBCH,(x424,)};{LTCBCH,(x385,)};{BCHETH,(x264,)};{ZECLTC,(x61,)}'

In [27]:
syntax = uvm2['rolling60'].reset_index()
syntax['syntax'] = '{' + syntax['index'] + ',(x' + syntax['rolling60'].astype(str) + ',)}'
syntaxprint = '(x1,);'+';'.join(syntax['syntax'])
syntaxprint

'(x1,);{ETHUSD,(x21388,)};{LTCUSD,(x33198,)};{LTCBTC,(x9176,)};{ZECUSD,(x10307,)};{ETHBTC,(x7806,)};{ZECBTC,(x3807,)};{BCHUSD,(x3107,)};{LTCETH,(x2473,)};{BTCUSD,(x2786,)};{BCHBTC,(x1807,)};{ZECETH,(x1001,)};{ZECBCH,(x504,)};{LTCBCH,(x385,)};{BCHETH,(x425,)};{ZECLTC,(x61,)}'

#### Result

In [2]:
# alertsnew = fixNum(unSMART(pd.read_excel('~/Documents/vmfldr/newcalibration_71.xlsx')))
# alertsnew2 = alertsnew.loc[alertsnew['Datetime'].dt.month > 3]
# pd.DataFrame(alertsnew2.groupby([alertsnew2['Datetime'].dt.month,'AlertCode','ShortText']).count()['Date'])

In [1]:
# alertsnew = fixNum(unSMART(pd.read_excel('~/Documents/vmfldr/smarts2.2_2019AprMay.xlsx')))
# alertsnew2 = alertsnew.loc[alertsnew['Datetime'].dt.month > 3]
# pd.DataFrame(alertsnew2.groupby([alertsnew2['Datetime'].dt.month,'AlertCode','ShortText']).count()['Date'])